In [1]:
cd ..

/Users/bastianlengen/Desktop/NameToBeFound


In [2]:
import pandas as pd
import numpy as np
import Fit_parameters as Fp

def load_data(work_dir='./'):
    '''
    Loads and return a dictionary of pandas DataFrame of the different .csv

    :type   work_dir: string
    :param  work_dir:
    '''
    Data_dir = work_dir + 'Data/'

    # Create an empty dictionnary of dataset
    DF_dict={}

    # Loads the Cepheids related DataFrame
    if Fp.include_Cepheids==True:
        print('Loading Cepheids & Cepheids anchors...')
        DF_dict['Cepheids'] = pd.read_csv(Data_dir+'Cepheids.csv', sep=',')
        DF_dict['Cepheids'].columns = ['Gal', 'logP', 'mW', 'sig_mW', 'M/H', 'z']
        DF_dict['Cepheids_anchors'] = pd.read_csv(Data_dir + 'Cepheids_anchors.csv', sep=',')
        DF_dict['Cepheids_anchors'].columns = ['Gal', 'logP', 'mW', 'sig_mW', 'M/H', 'z', 'mu', 'sig_mu']
        if Fp.include_MW==True:
            print('Loading MW Cepheids...')
            DF_dict['Cepheids_MW'] = pd.read_csv(Data_dir+'Cepheids_MW.csv', sep=',')
            DF_dict['Cepheids_MW'].columns = ['Gal', 'logP', 'mW', 'sig_mW', 'M/H', 'z', 'pi', 'sig_pi']
        print('Loading SNe for the Cepheids-host galaxies...')
        DF_dict['SNe_Cepheids'] = pd.read_csv(Data_dir + 'SNe_Cepheids.csv', sep=',')
        DF_dict['SNe_Cepheids'].columns = ['Gal', 'mB', 'sig_mB']

    # Loads the TRGB related DataFrame
    if Fp.include_TRGB==True:
        print('Loading TRGB & TRGB anchors...')
        DF_dict['TRGB'] = pd.read_csv(Data_dir+'TRGB.csv', sep=',')
        DF_dict['TRGB'].columns = ['Gal', 'm', 'sig_m', 'A', 'V-I', 'z']
        DF_dict['TRGB_anchors'] = pd.read_csv(Data_dir + 'TRGB_anchors.csv', sep=',')
        DF_dict['TRGB_anchors'].columns = ['Gal', 'm', 'sig_m', 'A', 'V-I', 'z', 'mu', 'sig_mu']
        print('Loading SNe for the TRGB-host galaxies...')
        DF_dict['SNe_TRGB'] = pd.read_csv(Data_dir + 'SNe_TRGB.csv', sep=',')
        DF_dict['SNe_TRGB'].columns = ['Gal', 'mB', 'sig_mB']

    # Loads the SNe for the aB fit
    if Fp.fit_aB==True:
        print('Loading SNe for the aB fit...')
        DF_dict['SNe_Hubble'] = pd.read_csv(Data_dir + 'SNe_Hubble.csv', sep=',')
        DF_dict['SNe_Hubble'].columns = ['name', 'mB', 'sig_mB', 'z', 'sig_z']

    return DF_dict

DF_dict = load_data()

Loading TRGB & TRGB anchors...
Loading SNe for the TRGB-host galaxies...
Loading SNe for the aB fit...


In [ ]:
def fit_distance_ladder(DF_dict, work_dir='./'):
    '''
    Return the raw result of the fit parameters

    :type   DF_dict: dictionary of pandas DataFrame
    :param  DF_dict: Dictionary that contains the DataFrame that will be fitted.
    :type   work_dir: string
    :param  work_dir: Working directory that contains the Data/ folder.
    '''
    Data_dir = work_dir + 'Data/'

    ### Useful functions
    # function that change a horizontal vector to a vertical vector
    def vert(v):
        v = np.array(v)
        return v.reshape(len(v),1)
    def logczexpansion(z):
        return np.log10(Fp.c*z*(1 + 1/2*(1-Fp.q0)*z - 1/6*(1-Fp.q0-3*Fp.q0**2+Fp.j0)*z**2))
    
    
    ### We first create an empty signal vector y, design matrix L and parameters vector q and Sigma
    [y, L, Sigma] = 3*[np.empty(0)]
    q_string = []
    
    # For the Cepheids
    if Fp.include_Cepheids == True:
        ### Load the DF
        # Use the Cepheids DF and check for its consistency with the Fit_parameters.py file
        Cepheids = DF_dict['Cepheids']
        galaxies_Cep = Cepheids['Gal'].drop_duplicates().reset_index(drop=True) # List of Cepheids-host galaxy
        if len(galaxies_Cep)!=Fp.N_galaxies_Cep:
            print('ERROR: make sure that the value for N_galaxies_Cep in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the Cepheids.csv file!')
            return
        # The anchors Cepheids
        Cepheids_anchors = DF_dict['Cepheids_anchors']
        anchors_Cep =  Cepheids_anchors['Gal'].drop_duplicates().reset_index(drop=True) # List of Cepheids-host galaxy
        if len(anchors_Cep)!=Fp.N_anchors_Cep:
            print('ERROR: make sure that the value for N_anchors_Cep in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the Cepheids_anchors.csv file!')
            return
        # The MW Cepheids
        if Fp.include_MW == True:
            Cepheids_MW = DF_dict['Cepheids_MW']  
        # The SNe from the Cepheid-host galaxy
        SNe_Cepheids = DF_dict['SNe_Cepheids']

        ### Complete y
        y = np.append(y, Cepheids['mW']) # Cepheids 
        y = np.append(y, Cepheids_anchors['mW']) # Cepheids anchors
        y = np.append(y, np.zeros([1,Fp.N_anchors_Cep])) # External constrainte (Delta mu_anchors = 0)
        # Milky Way Cepheids
        if Fp.include_MW == True:
            if Fp.fixed_zp == True:
                y = np.append(y, Cepheids_MW['mW'] \
                              - 10 + 5 * np.log10(Cepheids_MW['pi'])\
                              + 5/np.log(10)*Fp.zp/Cepheids_MW['pi'])
            else:
                y = np.append(y, Cepheids_MW['mW'] \
                              - 10 + 5 * np.log10(Cepheids_MW['pi']))
        if Fp.fixed_Zw == True:
            minus_MH = Cepheids['M/H']
            minus_MH = np.append(minus_MH, Cepheids_anchors['M/H'])
            minus_MH = np.append(minus_MH, np.zeros([1,Fp.N_anchors_Cep]))
            if Fp.include_MW == True:
                minus_MH = np.append(minus_MH, Cepheids_MW['M/H'])
            y = y-Fp.Zw*minus_MH
        y = np.append(y, SNe_Cepheids['mB'])

        ### Complete q_string for future dict so it's easier to access
        # The parameters will be in that ordre:
        # mu, MW, b, (bs), (zw), (zp)
        for gal in galaxies_Cep:
            q_string.append(f'mu_{gal}')
        for gal in anchors_Cep:
            q_string.append(f'Dmu_{gal}')
        q_string.append('MW')
        q_string.append('MB')
        if Fp.PLR_break == True:
            q_string.append('b_s')
            q_string.append('b_l')
        else:
            q_string.append('b')
        if Fp.fixed_Zw == False:
            q_string.append('Zw')
        if ((Fp.include_MW == True) and (Fp.fixed_zp==False)):
            q_string.append('zp')


        ### fill L
        # First create the matrix with just mu, Dmu, M, MB and b 
        L = np.zeros([len(y), len(q_string)])
        # Iterate over each line, start with Cepheids
        index_offset = 0
        for i in range(len(Cepheids)):
            # mu
            index = galaxies_Cep[galaxies_Cep==Cepheids.loc[i,'Gal']].index[0]
            L[i+index_offset,index] = 1
            # MW
            index = q_string.index('MW')
            L[i+index_offset,index] = 1
            # logP
            if Fp.PLR_break == False:
                index = q_string.index('b')
            else:
                threshold = np.log10(Fp.break_P)
                if Cepheids.loc[i, 'logP'] < threshold:
                    index = q_string.index('b_s')
                else:
                    index = q_string.index('b_l')
            L[i+index_offset,index] = Cepheids.loc[i,'logP']
            # Zw
            if Fp.fixed_Zw == False:
                index = q_string.index('Zw')
                L[i+index_offset,index] = Cepheids.loc[i,'M/H']
        # Anchors Cepheids
        index_offset += i+1
        for i in range(len(Cepheids_anchors)):
            # Dmu
            index = anchors_Cep[anchors_Cep==Cepheids_anchors.loc[i,'Gal']].index[0] + Fp.N_galaxies_Cep
            L[i+index_offset,index] = 1
            # MW
            index = q_string.index('MW')
            L[i+index_offset,index] = 1
            # logP
            if Fp.PLR_break == False:
                index = q_string.index('b')
            else:
                threshold = np.log10(Fp.break_P)
                if Cepheids_anchors.loc[i, 'logP'] < threshold:
                    index = q_string.index('b_s')
                else:
                    index = q_string.index('b_l')
            L[i+index_offset,index] = Cepheids_anchors.loc[i,'logP']
            # Zw
            if Fp.fixed_Zw == False:
                index = q_string.index('Zw')
                L[i+index_offset,index] = Cepheids_anchors.loc[i,'M/H']
        # External constriant (Delta mu)
        index_offset += i+1
        for i in range(Fp.N_anchors_Cep):
            L[i+index_offset,i+Fp.N_galaxies_Cep] = 1
        # MW Cepheids
        if Fp.include_MW == True:
            index_offset += i+1
            for i in range(len(Cepheids_MW)):
                # MW
                index = q_string.index('MW')
                L[i+index_offset,index] = 1
                # logP
                if Fp.PLR_break == False:
                    index = q_string.index('b')
                else:
                    threshold = np.log10(Fp.break_P)
                    if Cepheids_MW.loc[i, 'logP'] < threshold:
                        index = q_string.index('b_s')
                    else:
                        index = q_string.index('b_l')
                L[i+index_offset,index] = Cepheids_MW.loc[i,'logP']
                # Zw
                if Fp.fixed_Zw == False:
                    index = q_string.index('Zw')
                    L[i+index_offset,index] = Cepheids_MW.loc[i,'M/H']
                # zp
                if Fp.fixed_zp == False:
                    index = q_string.index('zp')
                    L[i+index_offset,index] = - 5/np.log(10)/Cepheids_MW.loc[i,'pi']
        # SNe from Cepheids-host galaxies
        index_offset += i+1
        for i in range(len(SNe_Cepheids)):
            # mu
            index = galaxies_Cep[galaxies_Cep==SNe_Cepheids.loc[i,'Gal']].index[0]
            L[i+index_offset,index] = 1
            # MB
            index = q_string.index('MB')
            L[i+index_offset,index] = 1
    
    # For the TRGB
    if Fp.include_TRGB == True:
        ### Load the DF
        # TRGB
        TRGB = DF_dict['TRGB']
        galaxies_TRGB = TRGB['Gal'].drop_duplicates().reset_index(drop=True) # List of TRGB-host galaxy
        if len(galaxies_TRGB)!=Fp.N_galaxies_TRGB:
            print('ERROR: make sure that the value for N_galaxies_TRGB in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the TRGB.csv file!')
        # TRGB anchors
        TRGB_anchors = DF_dict['TRGB_anchors']
        anchors_TRGB = TRGB_anchors['Gal'].drop_duplicates().reset_index(drop=True) # List of TRGB-host galaxy
        if len(anchors_TRGB)!=Fp.N_anchors_TRGB:
            print('ERROR: make sure that the value for N_anchors_TRGB in the Fit_parameters.py corresponds to \
                   the number of the galaxy in the TRGB_anchors.csv file!')
        # SNe_TRGB
        SNe_TRGB = DF_dict['SNe_TRGB']
            
        ### Complete y
        # TRGB
        y = np.append(y,TRGB['m']-TRGB['A'])
        # TRGB anchors
        y = np.append(y,TRGB_anchors['m']-TRGB_anchors['A'])
        if Fp.use_color == True:
            color = TRGB['V-I']-Fp.mid_VI
            color = np.append(color, TRGB_anchors['V-I']-Fp.mid_VI)
            y = y-0.2*color
        # SNe TRGB
        y = np.append(y,SNe_TRGB['mB'])
        
        ### Complete q
        # (Delta mu), mu, MTRGB
        if Fp.include_Cepheids == False:
            for gal in galaxies_TRGB:
                q_string.append(f'mu_{gal}')
                q_string.append('MB') # Have to declare it here if there is no Cepheid
        else:
            for gal in galaxies_TRGB:
                if gal in list(galaxies_Cep):
                    if Fp.different_mu == True:
                        q_string.append(f'Dmu_{gal}') # Delta mu if in both sets of galaxies
                else:
                    q_string.append(f'mu_{gal}')  
        q_string.append('MTRGB')
        if Fp.different_MB == True:
            q_string.append('DMB')
            
        ### Complete L
        # TRGB
        to_add_bot = np.zeros(len(TRGB),len(q_string))
        if Fp.include_Cepheids == False:
            #mu
            index = galaxies_TRGB[galaxies_TRGB==TRGB]

            
            # mu
            index = galaxies_Cep[galaxies_Cep==Cepheids.loc[i,'Gal']].index[0]
            L[i+index_offset,index] = 1
            # MW
            index = q_string.index('MW')
            L[i+index_offset,index] = 1
        

            
    # For the aB fit
    if Fp.fit_aB == True:
        ### Load the DF 
        SNe_Hubble = DF_dict['SNe_Hubble'] 
        SNe_Hubble = SNe_Hubble[(SNe_Hubble['z']>Fp.z_min)&(SNe_Hubble['z']<Fp.z_max)].reset_index(drop=True)
        
        ### Complete y
        y = np.append(y,SNe_Hubble['mB']-5*logczexpansion(SNe_Hubble['z'])-25)
        
        ### Complete q
        q_string.append('5logH0')
        
        ### Complete L 
        # First add a column for the extra parameters to the already existing matrice L
        L = np.hstack([L,np.zeros([len(L),1])])
        # Then add the lines
        to_add = np.zeros([len(SNe_Hubble), len(q_string)])
        # MB columns
        to_add[:,q_string.index('MB')] = 1
        # logH0 columns
        to_add[:,q_string.index('5logH0')] = -1
        L = np.vstack([L,to_add])



    # Solve
    
    q_dict = dict(zip(q_string,np.arange(len(q_string))))
    return y, q_string, L

y, q_string, L = fit_distance_ladder(DF_dict)

In [11]:
TRGB = DF_dict['TRGB']
galaxies_TRGB = TRGB['Gal'].drop_duplicates().reset_index(drop=True)
'M66' in list(galaxies_TRGB)

True

In [ ]:
dict(zip(q_string,L[975+209+2+67+19+1047]))

In [ ]:
DF_dict['SNe_Hubble']

In [13]:
A = []
np.vstack([A,[[1,2,3],[1,2,3]]])

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 0 and the array at index 1 has size 3